# Explore metadata fields in the full archaeology dataset

In the Notebook [collect_metadata.ipynb](collect_metadata.ipynb), I collected all metadata for the Archaeology Data Station through the API and saved it in a local MongoDB instance. 

The current Notebook contains code to get more insight into what's in the data. 

### Load data
Here we make a connection to the MongoDB database where the JSONs are saved. They include the full metadata of all datasets in the Archaeology Data Station. 

In [32]:
from pymongo import MongoClient

# Replace with your MongoDB connection string
CONNECTION_STRING = "mongodb://127.0.0.1:27018"  # For local MongoDB

# Connect to the database and collection
client = MongoClient(CONNECTION_STRING)
db = client["archaeology_metadata"] # Create new database
collection = db["collection"] # Create new collection

### Inspect data
Let's get an idea of what's in the metadata. First we count the total number of datasets, then we see many datasets have values for a selection of fields that may include geospatial data. 

In [53]:
count = collection.count_documents({})
print(f"Total documents in collection: {count}")

Total documents in collection: 158254


In [56]:
# This function counts the number of documents that have the specified field in the metadata

def get_count(path):

    """ 
    Print the count of the documents that have the specified field in the collection

    :param path: The path to the field in the document 
    """

    pipeline = [
        { "$match": { f"{path}": { "$exists": True, "$ne": None } } },
        { "$count": "total" }]  

    result = list(collection.aggregate(pipeline))
    count = result[0]["total"] if result else 0

    print(f"{path.split('.')[-1]}: {count}")

    #return count

In [findings.ipybn](findings.ipynb) you find a table with metadata fields that I identified earlier as possible containers of geospatial data in textual form. The cell below counts how many datasets have values in these fields. 

In [55]:
# Get the count of documents for various fields
get_count("ore:describes.dansTemporalSpatial:dansSpatialCoverageText")
get_count("ore:describes.dansTemporalSpatial:dansSpatialPoint")
get_count("ore:describes.dansTemporalSpatial:dansSpatialBox")
get_count("ore:describes.dansTemporalSpatial:dansSpatialCoverageControlled")
get_count("ore:describes.citation:dsDescription")
get_count("ore:describes.ore:aggregates.schema:name")
get_count("ore:describes.dansRelationMetadata:dansCollection")
get_count("ore:describes.citation:distributor") 



dansTemporalSpatial:dansSpatialCoverageText: 154396
dansTemporalSpatial:dansSpatialPoint: 56485
dansTemporalSpatial:dansSpatialBox: 4445
dansTemporalSpatial:dansSpatialCoverageControlled: 8274
citation:dsDescription: 158254
schema:name: 158253
dansRelationMetadata:dansCollection: 43907
citation:distributor: 50123


### Identify datasest with missing geospatial metadata
I assume that the coordinates would be present in either one of two fields: `dansSpatialPoint` or `dansSpatialBox`. These fields can have specifications like the X/Y coordinates, or North box. If you're reading this, and you know there's another field where coordinates can be present, please let me know. 

In [ ]:
# Count the number of documents where both geospatial fields are missing
query = {
    "$and": [
        {"ore:describes.dansTemporalSpatial:dansSpatialPoint": {"$exists": False}},
        {"ore:describes.dansTemporalSpatial:dansSpatialBox": {"$exists": False}}
    ]}

# Count the documents that match the query
missing_fields_count = collection.count_documents(query)

# Print the result
print(f" {missing_fields_count}")


 98412


For 98412 datasets, there is no value present in the 2 types of coordinate fields.

## Further inspect the datasets without geospatial data 

Let's extract the DOIs of some of the datasets with missing coordinates and manually inspect them. 


In [114]:
def select_values(field_path): 


    """
    Select the values of the specified field in the collection
    In this function, only the data that does not have coordinates is selected 

    :param field_path: The path to the field in the JSON
    :return: The values of the specified field
        
    """

    pipeline = [
    {
        "$match": {
            "$and": [
                {"ore:describes.dansTemporalSpatial:dansSpatialPoint": {"$exists": False}},
                {"ore:describes.dansTemporalSpatial:dansSpatialBox": {"$exists": False}}
            ]
        }
    },
    {
        "$project": {
            "_id": 0,
            f"{field_path}": 1
        }
    },
    {
        "$group": {
            "_id": None,
            "values": {"$push": f"${field_path}"}
        }
    },
    {
        "$project": {
            "_id": 0,
            "values": 1
        }
    }
    ]

    # Execute the aggregation pipeline
    result = collection.aggregate(pipeline)

    return result

In [109]:
# Extract the list of values
result = select_values("@id")
values_list = list(result)[0]['values'] if result else []


You can manually inspect the following 50 datasets to verify there's no coordinates in the metadata, and to look for other fields. 

In [116]:
# Print only the first 50 values
for url in values_list[:50]:
    print(url)

https://archaeology.datastations.nl/api/datasets/export?exporter=OAI_ORE&persistentId=doi:10.17026/dans-246-zkdb
https://archaeology.datastations.nl/api/datasets/export?exporter=OAI_ORE&persistentId=doi:10.17026/dans-zaz-bggb
https://archaeology.datastations.nl/api/datasets/export?exporter=OAI_ORE&persistentId=doi:10.17026/dans-xgk-zzky
https://archaeology.datastations.nl/api/datasets/export?exporter=OAI_ORE&persistentId=doi:10.17026/dans-zaa-rsht
https://archaeology.datastations.nl/api/datasets/export?exporter=OAI_ORE&persistentId=doi:10.17026/dans-x8b-vxz8
https://archaeology.datastations.nl/api/datasets/export?exporter=OAI_ORE&persistentId=doi:10.17026/dans-za2-essy
https://archaeology.datastations.nl/api/datasets/export?exporter=OAI_ORE&persistentId=doi:10.17026/dans-xrw-terb
https://archaeology.datastations.nl/api/datasets/export?exporter=OAI_ORE&persistentId=doi:10.17026/dans-z5j-bssu
https://archaeology.datastations.nl/api/datasets/export?exporter=OAI_ORE&persistentId=doi:10.170

### Inspect distributor 

In [120]:
distributors = select_values("ore:describes.citation:distributor")
distributors_list = list(distributors)[0]['values'] if distributors else [] 

In [143]:
from collections import Counter 

distribs = []

for dist in distributors_list:
    if type(dist) == dict:
        #print(list(dist.values())[0])
        distribs.append(list(dist.values())[0])
    else: 
        #print(dist)
        continue

# Count the number of occurrences of each distributor
distributors_count = Counter(distribs)

# Sort the distributors by the number of occurrences
sorted_dist = dict(sorted(distributors_count.items(), key=lambda item: item[1], reverse=True))
sorted_dist

{'Batavialand': 504,
 'RAAP Archeologisch Adviesbureau': 480,
 'Bilan, Fontys Hogescholen': 362,
 'Synthegra': 230,
 'De Steekproef, archeologisch onderzoeks- en adviesbureau': 163,
 'SOB Research': 151,
 'RAAP': 149,
 'Bureau Oudheidkundig Onderzoek van Gemeentewerken Rotterdam': 148,
 'Grontmij': 102,
 'Stichting RAAP': 90,
 'BAAC bv': 85,
 'Scarab': 81,
 'Artefact! Advies en Onderzoek in Erfgoed': 80,
 'Synthegra Archeologie bv': 68,
 'BAAC': 60,
 'Archeoplan Eco': 57,
 'Rijksmuseum van Oudheden': 56,
 'Stichting Raap': 51,
 'De Steekproef': 48,
 'ArchaeoBone': 46,
 'ROB': 42,
 'Leiden University Press': 36,
 'Rijksdienst voor het Cultureel Erfgoed': 35,
 'Historische Vereniging Holland': 34,
 'KSP Archeologie': 32,
 'Synthegra B.V.': 30,
 'ADC': 27,
 'MUG Ingenieursbureau': 24,
 'Hazenberg Archeologie': 23,
 'Econsultancy BV': 23,
 'ADC ArcheoProjecten': 20,
 'BAAC.BV': 20,
 'Synthegra bv': 18,
 'Jacobs en Burnier, archeologisch projectbureau': 18,
 'ROB, Amersfoort': 17,
 'BOOR': 

## Scrap
Below I put some code that I may use later

In [43]:
# Check counts of distinct values in the field "citation:distributor"
pipeline = [
    { "$group": { "_id": "$ore:describes.citation:distributor", "count": { "$sum": 1 } } },
    { "$sort": { "count": -1 } }
]

result = list(collection.aggregate(pipeline))
result = sorted(result, key=lambda x: x["count"], reverse=True)


In [ ]:
result